# Agent-based protein design, analysis and modeling 

In [1]:
import autogen
import os
import tqdm 
import time
import openai
from openai import AsyncOpenAI
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from IPython.display import Markdown, display
import py3Dmol

from tqdm import tqdm

from chroma import api
from chroma import Chroma, Protein, conditioners
from chroma.models import graph_classifier, procap
from chroma.utility.api import register_key
from chroma.utility.chroma import letter_to_point_cloud, plane_split_protein

chroma_key = '###'
HF_token= '###'
OpenAI_key='sk-###'
####################### CHROMA ##########################
api.register_key(chroma_key,)
chroma = Chroma()
###################### HF
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.embeddings import OpenAIEmbedding

token = HF_token
from huggingface_hub import login
login(token=token)

#pip install transformers==4.35.1

######### OPENAI ###########
os.environ['OPENAI_API_KEY']=OpenAI_key

Using cached data from /tmp/chroma_weights/90e339502ae6b372797414167ce5a632/weights.pt
Loaded from cache
Using cached data from /tmp/chroma_weights/03a3a9af343ae74998768a2711c8b7ce/weights.pt
Loaded from cache
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/alireza/.cache/huggingface/token
Login successful


### Using GPT-4 - Main setup starts here

In [2]:
device = "cpu"
import transformers
autogen.__version__, openai.__version__, transformers.__version__ #make sure to update both autogen and openai to their newsest versions 
# ('0.2.2', '1.6.0', '4.35.1')

('0.2.2', '1.6.0', '4.35.1')

In [3]:
from llm_config import llm_config
from llm_config import config_list
import agent_functions as func
import agents
import autogen

2023-12-31 14:05:52.867788: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-31 14:05:53.475945: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
#from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent

#config_list  = autogen.config_list_from_models(model_list=["gpt-4-1106-preview", "gpt-4", "gpt-3.5-turbo", "gpt-3.5-turbo-16k"])
#config_list  = autogen.config_list_from_models(model_list=["gpt-4-turbo", "gpt-4", "gpt-3.5-turbo", "gpt-3.5-turbo-16k"])
#llm_config={
#       # "request_timeout": 600,
#        "seed": 42,
#        "config_list": config_list,
#    }

# Construct the llm_config

'''
# Construct the llm_config
llm_config = {
  #Generate functions config for the Tool
  "config_list": config_list,  # Assuming you have this defined elsewhere
 # "request_timeout": 120,
}
'''
def _reset_agents():
    agents.user_proxy.reset()
    agents.ragproxyagent.reset()
    agents.planner.reset()
    agents.assistant.reset()
    agents.reviewer.reset()
    agents.coder.reset()
    agents.executor.reset()
    #sequence_retriever.reset()
#manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)
_reset_agents()

manager_llm_config = {
    "config_list": config_list,
    "seed": 45,
}

groupchat = autogen.GroupChat(
    agents=[agents.user_proxy, agents.planner, agents.assistant, agents.critic, #agents.coder, agents.executor  #sequence_retriever,
               ], messages=[], max_round=150, 
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config = manager_llm_config, 
                system_message='This agent repeats the following steps: Dynamically selecting a speaker, collecting response, and \
broadcasting the message to the group.')

In [5]:
agents.user_proxy.initiate_chat(
    recipient=manager, clear_history=True,
    message='''
    Design two proteins of length 100 for each family of mainly beta, mainly alpha, and alpha beta proteins.
    Then compute the maximum force and unfolding energy, analyze the secondary structure and 
    compute the first 10 frequencies for these proteins.
    '''
)

user_proxy (to chat_manager):


    Design two proteins of length 100 for each family of mainly beta, mainly alpha, and alpha beta proteins.
    Then compute the maximum force and unfolding energy, analyze the secondary structure and 
    compute the first 10 frequencies for these proteins.
    

--------------------------------------------------------------------------------
Planner (to chat_manager):

Plan 1:
1. Use the "design_protein_from_CATH" function with inputs of length of 100, CATH_ANNOTATION of 1 (mainly alpha), and a protein name for the first mainly alpha protein. Repeat with a different name for the second protein.
2. Use the same function again with inputs of length 100, CATH_ANNOTATION of 2 (mainly beta), and a protein name for the first mainly beta protein. Repeat with a different name for the second protein.
3. Use the function once more with inputs of length 100, CATH_ANNOTATION of 3 (alpha beta), and a protein name for the first alpha beta protein. Repeat with a dif

Integrating SDE:   0%|          | 0/300 [00:00<?, ?it/s]

Potts Sampling:   0%|          | 0/500 [00:00<?, ?it/s]

Sequential decoding:   0%|          | 0/100 [00:00<?, ?it/s]

assistant (to chat_manager):

***** Response from calling function "design_protein_from_CATH" *****
('./code_protein/mainly_alpha_protein_1.pdb', 'SMKKIEDYIREKLKALGLSDEEIEERVKQLMEGIKNPKKFEKELQKRNDRESLLIFKEAYALYEASKDKEKGKKLINKVQSERDKWETEQAEAARAAAAA')
*********************************************************************

--------------------------------------------------------------------------------
assistant (to chat_manager):

***** Suggested function Call: design_protein_from_CATH *****
Arguments: 

{
  "name": "mainly_alpha_protein_2",
  "CATH_ANNOTATION": "1",
  "length": 100
}
*************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION design_protein_from_CATH...
Using cached data from /tmp/chroma_weights/90e339502ae6b372797414167ce5a632/weights.pt
Loaded from cache
Using cached data from /tmp/chroma_weights/03a3a9af343ae74998768a2711c8b7ce/weights.pt
Loaded from cache


...

Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: Thanks.
user_proxy (to chat_manager):

Thanks.

--------------------------------------------------------------------------------


TERMINATE


Planner (to chat_manager):

You're welcome! If you have any other questions, feel free to ask.

--------------------------------------------------------------------------------


TERMINATE


assistant (to chat_manager):

TERMINATE

--------------------------------------------------------------------------------


In [6]:
agents.user_proxy.send(
    recipient=manager,
    message='''
Could you save the results in a CSV file named "protein_analysis.csv," 
where each row corresponds to a protein, and each column represents a specific property?
Include the AA sequence in the results and use suitable identifiers for the columns.    '''
)

user_proxy (to chat_manager):


Could you save the results in a CSV file named "protein_analysis.csv," 
where each row corresponds to a protein, and each column represents a specific property?
Include the AA sequence in the results and use suitable identifiers for the columns.    

--------------------------------------------------------------------------------
Planner (to chat_manager):

***** Suggested function Call: save_to_csv_file *****
Arguments: 

{
  "input_JSON_dictionary" : "{\"Protein Name\": [\"mainly_alpha_protein_1\", \"mainly_alpha_protein_2\", \"mainly_beta_protein_1\", \"mainly_beta_protein_2\", \"alpha_beta_protein_1\", \"alpha_beta_protein_2\"], \"AA Sequence\": [\"SMKKIEDYIREKLKALGLSDEEIEERVKQLMEGIKNPKKFEKELQKRNDRESLLIFKEAYALYEASKDKEKGKKLINKVQSERDKWETEQAEAARAAAAA\", \"MSKKEIEELKKKLDEIVETLKEYARQGDDACKKAADLIEEVKKALEEGNPEKYSQLKKKLTDAINKAIEEYRKRFEAEGKPEEAQKVIDKLKKILDEITN\", \"TTVTVTPPVADADGNEHSTVTAYGNKVTITITCPSNCTVTETVDGVAKTLGTVSGNQTITETRTIAPDEVVTRTYTCTPNASATSSKTQTVTIKG

TERMINATE


Planner (to chat_manager):

You're welcome! If you have any other questions or need further assistance, feel free to ask.

--------------------------------------------------------------------------------


TERMINATE


assistant (to chat_manager):

TERMINATE

--------------------------------------------------------------------------------


In [7]:
agents.user_proxy.send(
    recipient=manager,
    message='''
Based on the results of this example, can you check if the protein generator (Chroma) has been successful in creating proteins with desired structure?   '''
)

user_proxy (to chat_manager):


Based on the results of this example, can you check if the protein generator (Chroma) has been successful in creating proteins with desired structure?   

--------------------------------------------------------------------------------
Critic (to chat_manager):

Based on our results it seems that the Chroma protein generator was successful in creating proteins with desired secondary structure content that generally correspond to their CATH annotations:
- Both of the mainly alpha proteins showed a very high percentage (around 89%) of alpha-helix secondary structure ('H'), which is in line with their designation.
- The mainly beta proteins showed higher percentages of extended strand/beta-sheet secondary structure ('E'). Though, the percentages varied quite a bit (64% for mainly_beta_protein_1 and only 8% for mainly_beta_protein_2), which could be due to the complex nature of beta-structures.
- As for the alpha beta proteins, both showed both helix ('H') a

TERMINATE


Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: Thanks.
user_proxy (to chat_manager):

Thanks.

--------------------------------------------------------------------------------


TERMINATE


Planner (to chat_manager):

You're welcome! If you have any other questions or need further assistance, feel free to ask.

--------------------------------------------------------------------------------


TERMINATE


assistant (to chat_manager):

TERMINATE

--------------------------------------------------------------------------------


## Set up/test Chroma Protein generator